<a href="https://colab.research.google.com/github/allmore0/app_pprototipico/blob/main/Proyecto_Protot%C3%ADpico_401.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install streamlit
!pip install pymongo
import streamlit as st
import pandas as pd
from pymongo import MongoClient
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import plotly.express as px
# -----------------------------
# Configuración del modelo
# -----------------------------
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
      outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).numpy()

# -----------------------------
# Interfaz Streamlit
# -----------------------------
st.title("?? Chatbot de Reclutamiento")
#st.image("avatar.png", width=150)

menu = st.sidebar.selectbox("Menú", ["Subir CV", "Evaluar Candidatos", "Reporte"])

if menu == "Subir CV":
  st.header("?? Subir CV")
  candidate_id = st.text_input("ID del candidato")
  cv_text = st.text_area("Texto del CV")
  if st.button("Guardar CV"):
    cv_collection.insert_one({"candidate_id": candidate_id, "cv_text": cv_text})
    st.success(f"CV de {candidate_id} guardado correctamente.")

elif menu == "Evaluar Candidatos":
  st.header("?? Evaluar Candidatos")
  job_description = st.text_area("Descripción del puesto")
  if st.button("Evaluar"):
    cvs = list(cv_collection.find({}, {"_id": 0}))
    df = pd.DataFrame(cvs)
    similarities = []
    job_emb = get_embedding(job_description)
    for cv in df["cv_text"]:
      cv_emb = get_embedding(cv)
      sim = cosine_similarity(job_emb, cv_emb)[0][0]
      similarities.append(sim)
    df["similarity"] = similarities
    ranked_df = df.sort_values(by="similarity", ascending=False)
    st.write("Ranking de candidatos:")
    st.dataframe(ranked_df)

    # Gráfico de barras
    fig_bar = px.bar(ranked_df, x="candidate_id", y="similarity", title="Ranking por Similaridad")
    st.plotly_chart(fig_bar)

    # Botón para exportar ranking
    csv = ranked_df.to_csv(index=False).encode('utf-8')
    st.download_button(
      label="?? Descargar Ranking en CSV",
      data=csv,
      file_name="ranking_candidatos.csv",
      mime="text/csv"
    )

elif menu == "Reporte":
  st.header("?? Reporte para Reclutadores")
  cvs = list(cv_collection.find({}, {"_id": 0}))
  df = pd.DataFrame(cvs)
  if not df.empty:
    st.write(f"Total candidatos: {len(df)}")
    st.write("IDs:", df["candidate_id"].tolist())

    # Gráfico de distribución
    if "similarity" in df.columns:
      fig_hist = px.histogram(df, x="similarity", nbins=10, title="Distribución de Similaridad")
      st.plotly_chart(fig_hist)

      # Gráfico de dispersión
      fig_scatter = px.scatter(df, x="candidate_id", y="similarity",
      title="Dispersión de Similaridad por Candidato",
      size="similarity", color="similarity",
      hover_data=["cv_text"])

      st.plotly_chart(fig_scatter)

  # Botón para exportar reporte
  csv = df.to_csv(index=False).encode('utf-8')
  st.download_button(
    label="?? Descargar Reporte en CSV",
    data=csv,
    file_name="reporte_candidatos.csv",
    mime="text/csv"
  )
else:
  st.warning("No hay datos disponibles.")

2025-11-25 14:43:23.592 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-25 14:43:23.593 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-25 14:43:23.594 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-25 14:43:23.594 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-25 14:43:23.596 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-25 14:43:23.596 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-25 14:43:23.598 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-25 14:43:23.600 Session state does not function when running a script without `streamlit run`
2025-11-25 14:43